In [1]:
using LinearAlgebraicRepresentation
Lar= LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
using SparseArrays
using DataStructures
using NearestNeighbors

In [8]:
import Pkg;
Pkg.add("Images")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed NaNMath ─────────── v1.0.0
   Installed FixedPointNumbers ─ v0.8.4
   Installed ColorTypes ──────── v0.10.12
   Installed Colors ──────────── v0.12.8
┌ Warning: The active manifest file at `/Users/panemiele/IdeaProjects/LocalCongruence.jl/LAR_Congruence/Manifest.toml` has an old format that is being maintained.
│ To update to the new format run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
└ @ Pkg.Types /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.7/Pkg/src/manifest.jl:287
    Updating `~/IdeaProjects/LocalCongruence.jl/LAR_Congruence/Project.toml`
  [916415d5] + Images v0.23.3
    Updating `~/IdeaProjects/LocalCongruence.jl/LAR_Congruence/Manifest.toml`
  [621f4979] + AbstractFFTs v1.1.0
  [13072b0f] + AxisAlgorithms v1.0.1
  [39de3d68] + AxisArrays v0.4.6
  [49dc2e85] + Calculus v0.5.1
  [aafaddc9] + CatIndices v0.2.2

 [13] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
    @ Base ./loading.jl:1318
 [14] top-level scope
    @ none:1
 [15] eval
    @ ./boot.jl:373 [inlined]
 [16] eval(x::Expr)
    @ Base.MainInclude ./client.jl:453
 [17] top-level scope
    @ none:1
in expression starting at /Users/panemiele/.julia/packages/Interpolations/y4lLj/src/lanczos/lanczos_opencv.jl:65
in expression starting at /Users/panemiele/.julia/packages/Interpolations/y4lLj/src/Interpolations.jl:1
ERROR: LoadError: Failed to precompile Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59] to /Users/panemiele/.julia/compiled/v1.7/Interpolations/jl_gnm6Xu.
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:33
  [2] compilecache(pkg::Base.PkgId, path::String, internal_stderr::IO, internal_stdout::IO, ignore_loaded_modules::Bool)
    @ Base ./loading.jl:1466

LoadError: Failed to precompile Images [916415d5-f1e6-5110-898d-aaa5f9f070e0] to /Users/panemiele/.julia/compiled/v1.7/Images/jl_IyNLjZ.

In [22]:
using Images
img = Images

┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1423
ERROR: LoadError: TypeError: in <:, expected Type, got Vararg
Stacktrace:
  [1] #s10#5
    @ ~/.julia/packages/StaticArrays/1g9bq/src/util.jl:20 [inlined]
  [2] var"#s10#5"(T::Any, ::Any, #unused#::Any)
    @ StaticArrays ./none:0
  [3] (::Core.GeneratedFunctionStub)(::Any, ::Vararg{Any})
    @ Core ./boot.jl:580
  [4] macro expansion
    @ ~/.julia/packages/StaticArrays/1g9bq/src/SArray.jl:49 [inlined]
  [5] (StaticArrays.SArray{Tuple{2, 8}})(x::Tuple{Int64, Int64, Float64, Float64, Int64, Int64, Float64, Float64, Int64, Int64, Float64, Float64, Int64, Int64, Float64, Float64})
    @ StaticArrays ~/.julia/packages/StaticArrays/1g9bq/src/SArray.jl:46
  [6] _SA_typed_hvcat(sa::Type, rows::NTuple{8, Int64}, xs::Tuple{Int64, Int64, Float64, Float64, Int64, Int64, Float64, Float64, Int64, Int64, Float64, Float64, Int64, Int64, Float64, Float64})
    @ StaticArrays ~/.julia/packages/StaticArrays/1g9b

LoadError: Failed to precompile Images [916415d5-f1e6-5110-898d-aaa5f9f070e0] to /Users/panemiele/.julia/compiled/v1.7/Images/jl_sK9EKw.

In [9]:
function vertCongruence(W)
	err, i, todelete, vclasses = 10^-5.2,1,[], []
	verts = convert(Lar.Points, W')
	kdtree = NearestNeighbors.KDTree(verts)
	newverts = zeros(Int, size(verts,2))
	for vi in 1:size(verts,2)
		if !(vi in todelete)
			nearvs = NearestNeighbors.inrange(kdtree, verts[:,vi], err)
			push!(vclasses,nearvs)
			newverts[nearvs] .= i
			nearvs = setdiff(nearvs, vi)
			todelete = union(todelete, nearvs)
			i += 1
		end
	end
	V = zeros(3,length(vclasses))
	for (k,class) in enumerate(vclasses)
			V[:,k] = sum(W[class,:],dims=1)/length(class)
	end
	return V, vclasses
end


function cellCongruenceAA(Delta,inclasses)
	cellarray = Lar.cop2lar(Delta)
	new_e = Array{Int64,1}(undef,size(Delta,2))
	for (k,class) in enumerate(inclasses)
		for e in class
			new_e[e] = k
		end
	end
  cells = [map(e->new_e[e], face) for face in cellarray]
  outclasses = DefaultOrderedDict{Array{Int64,1},Array{Int64,1}}([])
  for (k,face) in enumerate(cells)
    if outclasses[face] == []
    	outclasses[face] = [k]
    else
    	append!(outclasses[face],[k])
    end
  end
	FEnew = sort(collect(keys(outclasses)))
  outclasses = sort(collect(values(outclasses)))
  return FEnew,outclasses
end


function chainCongruenceAA(W, T)
	V, vclasses = vertCongruence(W)
	EV, eclasses = cellCongruenceAA(T[1],vclasses)
	FE, fclasses = cellCongruenceAA(T[2],eclasses)
	#@show size(V,2) - size(EV,1) + size(FE,1)
	return V,EV,FE
end

chainCongruenceAA (generic function with 1 method)

In [10]:
V,EV,FV=Lar.randomcubes(30,.00001)

W = convert(Lar.Points, V');
cop_EV = Lar.coboundary_0(EV::Lar.Cells);
cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells);

T = [cop_EV, cop_FE]

2-element Vector{SparseMatrixCSC{Int8, Int64}}:
 sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2  …  357, 351, 356, 358, 352, 355, 359, 352, 356, 360], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  237, 238, 238, 238, 239, 239, 239, 240, 240, 240], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 360, 240)
 sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5  …  176, 180, 177, 179, 177, 180, 178, 179, 178, 180], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  356, 356, 357, 357, 358, 358, 359, 359, 360, 360], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 180, 360)

# Nove matrici di adiacenza/incidenza

In [12]:
VV = [[k] for k=1:size(V,2)]

240-element Vector{Vector{Int64}}:
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]
 [10]
 [11]
 [12]
 [13]
 ⋮
 [229]
 [230]
 [231]
 [232]
 [233]
 [234]
 [235]
 [236]
 [237]
 [238]
 [239]
 [240]

In [23]:
EV

360-element Vector{Vector{Int64}}:
 [1, 2]
 [3, 4]
 [5, 6]
 [7, 8]
 [1, 3]
 [2, 4]
 [5, 7]
 [6, 8]
 [1, 5]
 [2, 6]
 [3, 7]
 [4, 8]
 [9, 10]
 ⋮
 [233, 234]
 [235, 236]
 [237, 238]
 [239, 240]
 [233, 235]
 [234, 236]
 [237, 239]
 [238, 240]
 [233, 237]
 [234, 238]
 [235, 239]
 [236, 240]

In [13]:
VE=EV'

1×360 adjoint(::Vector{Vector{Int64}}) with eltype LinearAlgebra.Adjoint{Int64, Vector{Int64}}:
 [1 2]  [3 4]  [5 6]  [7 8]  [1 3]  [2 4]  …  [234 238]  [235 239]  [236 240]

In [24]:
FV

180-element Vector{Vector{Int64}}:
 [1, 2, 3, 4]
 [5, 6, 7, 8]
 [1, 2, 5, 6]
 [3, 4, 7, 8]
 [1, 3, 5, 7]
 [2, 4, 6, 8]
 [9, 10, 11, 12]
 [13, 14, 15, 16]
 [9, 10, 13, 14]
 [11, 12, 15, 16]
 [9, 11, 13, 15]
 [10, 12, 14, 16]
 [17, 18, 19, 20]
 ⋮
 [225, 226, 227, 228]
 [229, 230, 231, 232]
 [225, 226, 229, 230]
 [227, 228, 231, 232]
 [225, 227, 229, 231]
 [226, 228, 230, 232]
 [233, 234, 235, 236]
 [237, 238, 239, 240]
 [233, 234, 237, 238]
 [235, 236, 239, 240]
 [233, 235, 237, 239]
 [234, 236, 238, 240]

In [16]:
VF=FV'

1×180 adjoint(::Vector{Vector{Int64}}) with eltype LinearAlgebra.Adjoint{Int64, Vector{Int64}}:
 [1 2 3 4]  [5 6 7 8]  [1 2 5 6]  [3 4 7 8]  [1 3 5 7]  …  [234 236 238 240]

In [14]:
EE = VE.*EV

360×360 Matrix{Int64}:
   5    11    17    23    7    10    19  …     707     710     713     716
  11    25    39    53   15    22    43       1647    1654    1661    1668
  17    39    61    83   23    34    67       2587    2598    2609    2620
  23    53    83   113   31    46    91       3527    3542    3557    3572
   7    15    23    31   10    14    26        944     948     952     956
  10    22    34    46   14    20    38  …    1414    1420    1426    1432
  19    43    67    91   26    38    74       2824    2836    2848    2860
  22    50    78   106   30    44    86       3294    3308    3322    3336
  11    23    35    47   16    22    40       1418    1424    1430    1436
  14    30    46    62   20    28    52       1888    1896    1904    1912
  17    37    57    77   24    34    64  …    2358    2368    2378    2388
  20    44    68    92   28    40    76       2828    2840    2852    2864
  29    67   105   143   39    58   115       4467    4486    4505    4524
  

In [18]:
EF = EV.*VF

360×180 Matrix{Matrix{Int64}}:
 [1 2 3 4; 2 4 6 8]                  …  [234 236 238 240; 468 472 476 480]
 [3 6 9 12; 4 8 12 16]                  [702 708 714 720; 936 944 952 960]
 [5 10 15 20; 6 12 18 24]               [1170 1180 1190 1200; 1404 1416 1428 1440]
 [7 14 21 28; 8 16 24 32]               [1638 1652 1666 1680; 1872 1888 1904 1920]
 [1 2 3 4; 3 6 9 12]                    [234 236 238 240; 702 708 714 720]
 [2 4 6 8; 4 8 12 16]                …  [468 472 476 480; 936 944 952 960]
 [5 10 15 20; 7 14 21 28]               [1170 1180 1190 1200; 1638 1652 1666 1680]
 [6 12 18 24; 8 16 24 32]               [1404 1416 1428 1440; 1872 1888 1904 1920]
 [1 2 3 4; 5 10 15 20]                  [234 236 238 240; 1170 1180 1190 1200]
 [2 4 6 8; 6 12 18 24]                  [468 472 476 480; 1404 1416 1428 1440]
 [3 6 9 12; 7 14 21 28]              …  [702 708 714 720; 1638 1652 1666 1680]
 [4 8 12 16; 8 16 24 32]                [936 944 952 960; 1872 1888 1904 1920]
 [9 18 27 36; 10 20 3

In [25]:
FE = EF'

180×360 adjoint(::Matrix{Matrix{Int64}}) with eltype LinearAlgebra.Adjoint{Int64, Matrix{Int64}}:
 [1 2; 2 4; 3 6; 4 8]                  …  [236 240; 472 480; 708 720; 944 960]
 [5 10; 6 12; 7 14; 8 16]                 [1180 1200; 1416 1440; 1652 1680; 1888 1920]
 [1 2; 2 4; 5 10; 6 12]                   [236 240; 472 480; 1180 1200; 1416 1440]
 [3 6; 4 8; 7 14; 8 16]                   [708 720; 944 960; 1652 1680; 1888 1920]
 [1 2; 3 6; 5 10; 7 14]                   [236 240; 708 720; 1180 1200; 1652 1680]
 [2 4; 4 8; 6 12; 8 16]                …  [472 480; 944 960; 1416 1440; 1888 1920]
 [9 18; 10 20; 11 22; 12 24]              [2124 2160; 2360 2400; 2596 2640; 2832 2880]
 [13 26; 14 28; 15 30; 16 32]             [3068 3120; 3304 3360; 3540 3600; 3776 3840]
 [9 18; 10 20; 13 26; 14 28]              [2124 2160; 2360 2400; 3068 3120; 3304 3360]
 [11 22; 12 24; 15 30; 16 32]             [2596 2640; 2832 2880; 3540 3600; 3776 3840]
 [9 18; 11 22; 13 26; 15 30]           …  [2124 2160; 25

In [17]:
FF= VF .* FV

180×180 Matrix{Int64}:
   30    70    44    64    50    60    110  …    2364    2384    2370    2380
   70   174   100   152   114   140    278       6132    6184    6146    6172
   44   100    66    94    74    88    156       3314    3342    3322    3336
   64   152    94   138   106   128    240       5198    5242    5210    5232
   50   114    74   106    84   100    178       3786    3818    3796    3812
   60   140    88   128   100   120    220  …    4728    4768    4740    4760
  110   278   156   240   178   220    446       9900    9984    9922    9964
  150   382   212   328   242   300    614      13668   13784   13698   13756
  124   308   178   270   202   248    492      10850   10942   10874   10920
  144   360   206   314   234   288    576      12734   12842   12762   12816
  130   322   186   282   212   260    514  …   11322   11418   11348   11396
  140   348   200   304   228   280    556      12264   12368   12292   12344
  190   486   268   416   306   380    78

# <br><br><br><br><br> Chain Congruence

In [26]:
V,VE,FV=chainCongruenceAA(W,T)

([1.1624662 1.1624753 … 1.102137575 1.10213355; 0.5304282499999999 0.5304296500000001 … 0.144851975 0.144851975; 1.000068 1.0000715 … 1.13912805 1.13913365], [[1, 1], [1, 2], [2, 2], [3, 3], [3, 4], [4, 4], [5, 5], [5, 6], [5, 7], [6, 6]  …  [89, 90], [89, 91], [90, 90], [90, 92], [91, 91], [91, 92], [92, 92], [93, 93], [93, 94], [94, 94]], [[1, 1, 1, 1], [1, 2, 3, 3], [2, 2, 2, 2], [4, 4, 4, 4], [4, 5, 6, 6], [5, 5, 5, 5], [7, 7, 9, 10], [7, 8, 13, 14], [8, 8, 11, 12], [9, 11, 13, 13]  …  [131, 133, 135, 137], [132, 134, 136, 138], [139, 139, 141, 142], [139, 140, 145, 146], [140, 140, 143, 144], [141, 143, 145, 145], [142, 144, 146, 146], [147, 147, 148, 149], [148, 148, 148, 148], [149, 149, 149, 149]])

In [27]:
VE

149-element Vector{Vector{Int64}}:
 [1, 1]
 [1, 2]
 [2, 2]
 [3, 3]
 [3, 4]
 [4, 4]
 [5, 5]
 [5, 6]
 [5, 7]
 [6, 6]
 [6, 8]
 [7, 7]
 [7, 8]
 ⋮
 [87, 88]
 [89, 89]
 [89, 90]
 [89, 91]
 [90, 90]
 [90, 92]
 [91, 91]
 [91, 92]
 [92, 92]
 [93, 93]
 [93, 94]
 [94, 94]

In [28]:
FV

120-element Vector{Vector{Int64}}:
 [1, 1, 1, 1]
 [1, 2, 3, 3]
 [2, 2, 2, 2]
 [4, 4, 4, 4]
 [4, 5, 6, 6]
 [5, 5, 5, 5]
 [7, 7, 9, 10]
 [7, 8, 13, 14]
 [8, 8, 11, 12]
 [9, 11, 13, 13]
 [10, 12, 14, 14]
 [15, 15, 16, 17]
 [16, 16, 16, 16]
 ⋮
 [128, 130, 137, 138]
 [129, 130, 133, 134]
 [131, 133, 135, 137]
 [132, 134, 136, 138]
 [139, 139, 141, 142]
 [139, 140, 145, 146]
 [140, 140, 143, 144]
 [141, 143, 145, 145]
 [142, 144, 146, 146]
 [147, 147, 148, 149]
 [148, 148, 148, 148]
 [149, 149, 149, 149]